In [7]:
import os
from dotenv import load_dotenv
load_dotenv()

True

## Loading Documnets


In [8]:
## Loading from text file

from langchain_community.document_loaders import TextLoader
loader = TextLoader('sample.txt')
text_document = loader.load()

In [17]:
## Loading from a URL

from langchain_community.document_loaders import WebBaseLoader
import bs4

webloader = WebBaseLoader(
    web_path="https://pytorch.org/tutorials/beginner/pytorch_with_examples.html",
    bs_kwargs=dict(parse_only=bs4.SoupStrainer(
        class_=('main-content')
    )
))
text_web = webloader.load()


In [27]:
## Loading from pdf 

from langchain_community.document_loaders import PyPDFLoader
pdf_loader = PyPDFLoader('attention.pdf')
pdf_text = pdf_loader.load()

## Transforming Documents

In [32]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

In [126]:
documents = text_splitter.split_documents(pdf_text)

In [127]:
f = open("output.txt", "w")
for doc in documents:
    f.write(str(doc))
    f.write("\n")
f.close()
print("Done")

Done


## Embeddings

In [128]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [129]:
# chroma_db = Chroma.from_documents(documents, embeddings) 
# this shits not working

In [130]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents, embeddings)

In [131]:
query = "Who are the authors?"
result = db.similarity_search(query)

In [132]:
db.save_local("faiss_index")

In [133]:
from langchain_google_genai import GoogleGenerativeAI

chatbot = GoogleGenerativeAI(model="gemini-1.5-pro-latest")

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
    Answer the following question based only on provided context. Think step by step before provinding an answer.
    <context>
        {context}
    </context>
    Question: {input}
""")

### Chains

In [134]:
from langchain.chains.combine_documents import create_stuff_documents_chain

chain = create_stuff_documents_chain(chatbot, prompt)

### Retriever

In [135]:
retriver = db.as_retriever()

In [136]:
from langchain.chains import create_retrieval_chain

retrievla_chain = create_retrieval_chain(retriver, chain)

In [140]:
result = retrievla_chain.invoke({"input": "What are the decoders and encoders ?"})

In [141]:
result['answer']

'Let\'s think step-by-step:\n\n* **The provided text describes the architecture of a "Transformer" model.**  These models are commonly used in sequence-to-sequence tasks like machine translation.\n* **The text states both the encoder and decoder are composed of stacks of identical layers.** This suggests they are key structural components of the model.\n* **The encoder\'s description focuses on processing input ("injecting information about the position") with no mention of output.** This hints at its role in understanding the input sequence.\n* **The decoder, unlike the encoder, has an added sub-layer for multi-head attention over the encoder\'s output.** This suggests it uses information extracted by the encoder to generate an output sequence. \n\n**Therefore, based on the context:**\n\n* **The encoder** is a stack of layers responsible for processing and understanding the input sequence.\n* **The decoder** is a stack of layers that takes the encoder\'s output and generates the outpu

In [142]:
result

{'input': 'What are the decoders and encoders ?',
 'context': [Document(page_content='connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,\nrespectively.\n3.1 Encoder and Decoder Stacks\nEncoder: The encoder is composed of a stack of N= 6 identical layers. Each layer has two\nsub-layers. The ﬁrst is a multi-head self-attention mechanism, and the second is a simple, position-\n2', metadata={'source': 'attention.pdf', 'page': 1}),
  Document(page_content='of the softmax which correspond to illegal connections. See Figure 2.\n3.3 Position-wise Feed-Forward Networks\nIn addition to attention sub-layers, each of the layers in our encoder and decoder contains a fully\nconnected feed-forward network, which is applied to each position separately and identically. This\nconsists of two linear transformations with a ReLU activation in between.\nFFN(x) = max(0,xW 1+b1)W2+b2 (2)\nWhile the linear transformations are the same across different positions, 